In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import wikipedia
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import pickle

In [78]:
def delete_endtext(tex: str):
    subtex = tex[0:tex.find('\n\n== Notes ==\n')]
    return subtex

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_presidents_of_the_United_States'
res = requests.get(url)
#print(res.status_code)

tableclass = 'wikitable sortable jquery-tablesorter'
soup = BeautifulSoup(res.text, 'html.parser')
tag = soup.find('table',{'class':'wikitable'})
table=pd.read_html(str(tag))[0]

path = 'data/texts.pkl'
with open(path, 'wb') as f:
    pickle.dump(texts,f)

In [4]:
path = 'data/table.pkl'
f = open(path, 'rb')
table = pickle.load(f)

In [5]:
presidents = table.drop(columns=['Portrait','Party[b][15]'])
presidents = presidents.replace('[*]','')

presidents.columns = presidents.columns.str.replace(r'(\[\S+\])','')
presidents.columns = presidents.columns.str.replace(r'(\(\S+\))','')

presidents.rename(columns={'Party.1':'Party'}, inplace = True)
presidents.insert(2,'Birth - Death','')
presidents.set_index('No.', inplace = True)

presidents = presidents.apply(lambda row: row.str.replace(r'(\[\S+\])',''))

presidents['Birth - Death'] = presidents['Name'].str.extract(r'(\d+\S\d+)')
presidents['Name'] = presidents['Name'].str.replace(r'(\(\d+\S\d+\))','')
presidents['Name'] = presidents['Name'].str.replace(r'(\(b. \d+\))','')

In [11]:
presidents.head()

,Name,Birth - Death,Term,Party,Election,Vice President
No.,,,,,,
1,George Washington,1732–1799,"April 30, 1789–March 4, 1797",Unaffiliated,1788–1789 1792,John Adams
2,John Adams,1735–1826,"March 4, 1797–March 4, 1801",Federalist,1796,Thomas Jefferson
3,Thomas Jefferson,1743–1826,"March 4, 1801–March 4, 1809",Democratic-Republican,1800 1804,Aaron Burr George Clinton
4,James Madison,1751–1836,"March 4, 1809–March 4, 1817",Democratic-Republican,1808 1812,"George Clinton Vacant afterApril 20, 1812 Elbr..."
5,James Monroe,1758–1831,"March 4, 1817–March 4, 1825",Democratic-Republican,1816 1820,Daniel D. Tompkins


In [31]:
pres_list = presidents['Name'].drop_duplicates().str.replace(' ','_').to_list()
texts = pd.Series(index = pres_list, dtype = 'object')

#next_pres = 'Lyndon_B._Johnson'
#for pres in pres_list[pres_list.index(next_pres):]:
for pres in pres_list:
    wikisearch = wikipedia.page(pres, auto_suggest = False)
    texts[pres] = wikisearch.content
    print(pres)

George_Washington
John_Adams
Thomas_Jefferson
James_Madison
James_Monroe
John_Quincy_Adams
Andrew_Jackson
Martin_Van_Buren
William_Henry_Harrison
John_Tyler
James_K._Polk
Zachary_Taylor
Millard_Fillmore
Franklin_Pierce
James_Buchanan
Abraham_Lincoln
Andrew_Johnson
Ulysses_S._Grant
Rutherford_B._Hayes
James_A._Garfield
Chester_A._Arthur
Grover_Cleveland
Benjamin_Harrison
William_McKinley
Theodore_Roosevelt
William_Howard_Taft
Woodrow_Wilson
Warren_G._Harding
Calvin_Coolidge
Herbert_Hoover
Franklin_D._Roosevelt
Harry_S._Truman
Dwight_D._Eisenhower
John_F._Kennedy
Lyndon_B._Johnson
Richard_Nixon
Gerald_Ford
Jimmy_Carter
Ronald_Reagan
George_H._W._Bush
Bill_Clinton
George_W._Bush
Barack_Obama
Donald_Trump
Joe_Biden


In [35]:
path = 'data/texts.pkl'
with open(path, 'wb') as f:
    pickle.dump(texts,f)

In [57]:
path = 'data/texts.pkl'
f = open(path, 'rb')
rawtexts = pickle.load(f)

In [33]:
texts.index

Index(['George_Washington', 'John_Adams', 'Thomas_Jefferson', 'James_Madison',
       'James_Monroe', 'John_Quincy_Adams', 'Andrew_Jackson',
       'Martin_Van_Buren', 'William_Henry_Harrison', 'John_Tyler',
       'James_K._Polk', 'Zachary_Taylor', 'Millard_Fillmore',
       'Franklin_Pierce', 'James_Buchanan', 'Abraham_Lincoln',
       'Andrew_Johnson', 'Ulysses_S._Grant', 'Rutherford_B._Hayes',
       'James_A._Garfield', 'Chester_A._Arthur', 'Grover_Cleveland',
       'Benjamin_Harrison', 'William_McKinley', 'Theodore_Roosevelt',
       'William_Howard_Taft', 'Woodrow_Wilson', 'Warren_G._Harding',
       'Calvin_Coolidge', 'Herbert_Hoover', 'Franklin_D._Roosevelt',
       'Harry_S._Truman', 'Dwight_D._Eisenhower', 'John_F._Kennedy',
       'Lyndon_B._Johnson', 'Richard_Nixon', 'Gerald_Ford', 'Jimmy_Carter',
       'Ronald_Reagan', 'George_H._W._Bush', 'Bill_Clinton', 'George_W._Bush',
       'Barack_Obama', 'Donald_Trump', 'Joe_Biden'],
      dtype='object')

In [58]:
type(rawtexts)

pandas.core.series.Series

In [38]:
texts['Theodore_Roosevelt'].shape

(59,)

In [40]:
texts['Theodore_Roosevelt']

0     Theodore  Roosevelt Jr. ( ROH-zə-velt; October...
1      \nTheodore Roosevelt Jr. was born on October ...
2     Roosevelt was homeschooled, mostly by tutors a...
3     While at Harvard, Roosevelt began a systematic...
4     In 1880, Roosevelt married socialite Alice Hat...
5     Roosevelt was a member of the New York State A...
6     With numerous presidential hopefuls from whom ...
7     Roosevelt first visited the Dakota Territory i...
8     On December 2, 1886, Roosevelt married his chi...
9     Upon Roosevelt's return to New York in 1886, R...
10    After Benjamin Harrison unexpectedly defeated ...
11    In 1894, a group of reform Republicans approac...
12    In the 1896 presidential election, Roosevelt b...
13    With the beginning of the Spanish–American War...
14    After leaving Cuba in August 1898, the Rough R...
15    In November 1899, Vice President Garret Hobart...
16    On September 6, 1901, President McKinley was a...
17    For his aggressive use of the 1890 Sherman

In [53]:
texts.index['Thomas_Jefferson':'James_Monroe']

TypeError: slice indices must be integers or None or have an __index__ method

In [77]:
p = ['William_Henry_Harrison']
#for pres in rawtexts.index:
#for pres in rawtexts.index[rawtexts.index.get_loc('Theodore_Roosevelt'):]:
for pres in p: 
    rawtexts[pres] = delete_endtext(rawtexts[pres])
    sections = re.split('\n==.+==\n',rawtexts[pres])
    texts[pres] = pd.Series(data = sections, name = pres)
    texts[pres] = texts[pres][texts[pres] != '\n'].reset_index(drop = True)

    for sect in range(texts[pres].shape[0]):
        if texts[pres][sect] == '':
            texts[pres] = texts[pres].drop(sect)
        else:
            char_1 = texts[pres][sect][0]
            if (char_1 == '\n'):
                texts[pres][sect] = texts[pres][sect][1:]

-1


In [ ]:
\n\n== Notes ==\n

In [143]:
uneek = pd.Series(index = pres_list, dtype='Int64')

for pres in uneek.index:
    uneek[pres] = len(set(texts[pres].sum().split(' ')))

In [147]:
print(uneek.idxmin(),uneek.min(),uneek.idxmax(),uneek.max())

Zachary_Taylor 2726 Theodore_Roosevelt 6049


In [149]:
tot_text = texts.sum()